<h1 align="center">
<img src="https://drive.google.com/uc?export=view&id=1_ZkE3dnYg9KykzdZE6lIK2s_q_nbiKk_" width="160" height="160">
</h1>
<h1 align="center">Massive Data Analytics - 25577</h1>
<h3 align="center">Dr. Gholampour</h3>
<h5 align="center">Sharif University of Technology<br/>Electrical Engineering Department<br/>  Fall 2022</h5>

---

# Problem Set 03
<h4 align="center">Vahid Pourakbar     400205511</h4>

# Question 3

In [1]:
%reset -f
import numpy as np
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("MDA_2022_PS3_Q3") \
    .master("local[*]") \
    .config("spark.driver.memory", "10g") \
    .getOrCreate()
    # .config("spark.memory.offHeap.enabled","true") \
    # .config("spark.memory.offHeap.size","4g")  \
sc=spark.sparkContext

23/01/23 22:32:20 WARN Utils: Your hostname, vahid-PC resolves to a loopback address: 127.0.1.1; using 172.27.211.114 instead (on interface wlp2s0)
23/01/23 22:32:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/23 22:32:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
dataframe = spark.read.csv("SampleData.csv", header=True, inferSchema=True)
dataframe.limit(5).show()

+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|DEVICE_CODE|SYSTEM_ID|ORIGINE_CAR_KEY|FINAL_CAR_KEY|CHECK_STATUS_KEY|COMPANY_ID|      PASS_DAY_TIME|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|  100700853|      283|        9016704|      9016704|               5|       264|2022-01-08 00:00:39|
|     230204|       81|        9017538|      9017538|               5|       161|2022-01-08 00:38:48|
|     631357|       81|        9017538|      9017538|               5|       161|2022-01-08 00:13:50|
|     900164|      283|        9035020|      9035020|               5|       264|2022-01-08 00:27:38|
|     900149|      283|        9020900|      9020900|               5|       264|2022-01-08 00:40:08|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+



## Part A

In [3]:
dataframe_rdd = dataframe["DEVICE_CODE","FINAL_CAR_KEY", "PASS_DAY_TIME"].rdd \
    .map(lambda x: ((x[1], x[2].date()), [x[0]])) \
        .reduceByKey(lambda x, y: x+y)

print("key = (plate, date), value=[list of device codes]")
dataframe_rdd.take(10)

key = (plate, date), value=[list of device codes]


[((9021057, datetime.date(2022, 1, 8)), [900101]),
 ((9026889, datetime.date(2022, 1, 8)), [900266]),
 ((9197057, datetime.date(2022, 1, 8)), [100700853, 22010078, 22010079]),
 ((9148073, datetime.date(2022, 1, 8)),
  [900202,
   900244,
   900199,
   900264,
   900213,
   900212,
   631633,
   107,
   900244,
   900222,
   900101,
   900213,
   900212,
   200801,
   200502,
   900243]),
 ((9150129, datetime.date(2022, 1, 8)), [900158]),
 ((9240801, datetime.date(2022, 1, 8)),
  [205202, 900126, 205201, 100701250, 900241, 169]),
 ((9235577, datetime.date(2022, 1, 8)), [900266]),
 ((9350137, datetime.date(2022, 1, 8)), [900101, 100700841]),
 ((9354665, datetime.date(2022, 1, 8)),
  [900212, 900264, 900156, 900101, 900156]),
 ((9387345, datetime.date(2022, 1, 8)), [230107])]

## Part B

In [4]:
import math 
random_path = [205202, 900126, 205201, 100701250, 900241, 169]
top_five_similar_pathes_rdd = dataframe_rdd.map(lambda x: ((x[0][0], x[1]), len(set(random_path)&set(x[1]))/math.sqrt(len(set(random_path))*len(set(x[1]))) )) \
    .takeOrdered(5, key = lambda x: -x[1])
print(f"random path = [205202, 900126, 205201, 100701250, 900241, 169],\n \t ((plate, path), similarity):", end='')
top_five_similar_pathes_rdd

random path = [205202, 900126, 205201, 100701250, 900241, 169],
 	 ((plate, path), similarity):

[((9240801, [205202, 900126, 205201, 100701250, 900241, 169]), 1.0),
 ((12169977, [169, 169, 205201, 205202, 900126, 900197, 900241, 103001]),
  0.7715167498104595),
 ((13726086,
   [100701155, 205201, 900241, 205202, 100701251, 169, 100701250, 141, 169]),
  0.7216878364870323),
 ((7867577, [169, 205202, 205201]), 0.7071067811865476),
 ((89049300, [900126, 900126, 900126, 169, 900241]), 0.7071067811865476)]

## Part C

In [5]:
cameras_list = dataframe_rdd.flatMap(lambda x: x[1]) \
    .distinct() \
        .collect()
cameras_list = sorted(cameras_list)
# sc.broadcast(cameras_list)
camera_list_count = len(cameras_list)
print(f"number of distinct camera: {camera_list_count}")

number of distinct camera: 922


In [6]:
def generating_random_hyperplane(NumberOfRandomHyperplane):
    random_hyperplanes_list = []
    while (len(random_hyperplanes_list) < NumberOfRandomHyperplane):
        randlist = np.random.choice([-1, 1], size=camera_list_count)
        flag = 1
        for l in random_hyperplanes_list:
            if all(randlist==l):
                flag = 0
                break
        if flag:
            random_hyperplanes_list.append(randlist)
    return random_hyperplanes_list

random_hyperplanes_list = generating_random_hyperplane(NumberOfRandomHyperplane=20)

In [7]:
def generating_signature_matrix(input):
    signature_list = []
    input_camera_list = sorted(input)
    camera_index = [cameras_list.index(c) for c in input_camera_list]
    for random_hyperplane in random_hyperplanes_list:
        rh = sum([random_hyperplane[idx] for idx in camera_index])
        signature_list.append(1 if rh>=0 else -1)
    return signature_list

def Or_And(input):
    return all([any([x==y for x, y in zip(input[bb*r:(bb+1)*r],random_path_sig_mat[bb*r:(bb+1)*r])]) for bb in range(b)])
def And_Or(input):
    return any([all([x==y for x, y in zip(input[bb*r:(bb+1)*r],random_path_sig_mat[bb*r:(bb+1)*r])]) for bb in range(b)])

In [ ]:
# def func(input):
#     signature_list = []
#     input_camera_list = sorted(input)
#     camera_boolen = [int(l in input_camera_list) for l in cameras_list]

#     for random_hyperplane in random_hyperplanes_list:
#         rh = sum(np.array(random_hyperplane)*np.array(camera_boolen))
#         signature_list.append(1 if rh>=0 else -1)
#     return signature_list

# signature_matrix_rdd = dataframe_rdd.map(lambda x: ((x[0][0], x[1]), func(x[1])))
# signature_matrix = signature_matrix_rdd.collect()
# 7min for 6 hyperplane
# >30min for 60 hyperplane

In [8]:
signature_matrix_rdd = dataframe_rdd.map(lambda x: ((x[0][0], x[1]), generating_signature_matrix(x[1])))
signature_matrix = signature_matrix_rdd.collect()
len(signature_matrix)

1479063

In [9]:
random_path_sig_mat = generating_signature_matrix(random_path)
print(random_path_sig_mat)

[1, 1, 1, 1, -1, -1, -1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1]


In [10]:
# r*b=NumberOfRandomHyperplane
r = 5
b = 4

similarity_candidates_rdd = signature_matrix_rdd.map(lambda x: (x[0], And_Or(x[1]))) \
    .filter(lambda x: x[1])
similarity_candidates = similarity_candidates_rdd.collect()
print(len(similarity_candidates))
similarity_candidates_rdd.take(2)


248168


[((9150129, [900158]), True),
 ((9240801, [205202, 900126, 205201, 100701250, 900241, 169]), True)]

In [11]:
similarity_candidates_rdd_2 = signature_matrix_rdd.map(lambda x: (x[0], Or_And(x[1]))) \
    .filter(lambda x: x[1])
similarity_candidates_2 = similarity_candidates_rdd_2.collect()
print(len(similarity_candidates_2))
similarity_candidates_rdd_2.take(2)

1301623


[((9021057, [900101]), True), ((9026889, [900266]), True)]

## Number of random hyperplane =  40

In [82]:
random_hyperplanes_list = generating_random_hyperplane(NumberOfRandomHyperplane=40)

In [13]:
signature_matrix_rdd_40 = dataframe_rdd.map(lambda x: ((x[0][0], x[1]), generating_signature_matrix(x[1])))
signature_matrix_40 = signature_matrix_rdd_40.collect()
len(signature_matrix_40)

1479063

In [83]:
random_path_sig_mat = generating_signature_matrix(random_path)
print(random_path_sig_mat)

[1, 1, 1, 1, 1, -1, 1, 1, 1, -1, -1, -1, -1, -1, 1, -1, 1, -1, 1, -1, -1, -1, 1, 1, -1, 1, -1, -1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, -1]


In [15]:
# r*b=NumberOfRandomHyperplane
r = 8
b = 5

similarity_candidates_rdd_40 = signature_matrix_rdd_40.map(lambda x: (x[0], And_Or(x[1]))) \
    .filter(lambda x: x[1])
similarity_candidates_40 = similarity_candidates_rdd_40.collect()
print(len(similarity_candidates_40))
similarity_candidates_rdd_40.take(2)

49084


[((9240801, [205202, 900126, 205201, 100701250, 900241, 169]), True),
 ((9847265, [145, 205802, 175]), True)]

In [84]:
# r*b=NumberOfRandomHyperplane
r = 5
b = 8

similarity_candidates_rdd_40_2 = signature_matrix_rdd_40.map(lambda x: (x[0], Or_And(x[1]))) \
    .filter(lambda x: x[1])
similarity_candidates_40_2 = similarity_candidates_rdd_40_2.collect()
print(len(similarity_candidates_40_2))
similarity_candidates_rdd_40_2.take(2)

1122900


[((9021057, [900101]), True), ((9026889, [900266]), True)]

## Number of random hyperplane =  60

In [88]:
random_hyperplanes_list = generating_random_hyperplane(NumberOfRandomHyperplane=60)

In [17]:
signature_matrix_rdd_60 = dataframe_rdd.map(lambda x: ((x[0][0], x[1]), generating_signature_matrix(x[1])))
signature_matrix_60 = signature_matrix_rdd_60.collect()
len(signature_matrix_60)

1479063

In [87]:
random_path_sig_mat = generating_signature_matrix(random_path)
print(random_path_sig_mat)

[1, -1, -1, 1, -1, 1, 1, 1, 1, 1, -1, -1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, -1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, -1, -1, 1, 1]


In [19]:
# r*b=NumberOfRandomHyperplane
r = 10
b = 6

similarity_candidates_rdd_60 = signature_matrix_rdd_60.map(lambda x: (x[0], And_Or(x[1]))) \
    .filter(lambda x: x[1])
similarity_candidates_60 = similarity_candidates_rdd_60.collect()
print(len(similarity_candidates_60))
similarity_candidates_rdd_60.take(2)

17424


[((9240801, [205202, 900126, 205201, 100701250, 900241, 169]), True),
 ((8622505, [900225]), True)]

In [89]:
r1, b1, r2, b2 = 2, 3, 2, 5
def And_Or_Or_And(input):
    rb_list = []
    for r, b in zip(np.array(input).reshape(b2,r2,b1,r1), np.array(random_path_sig_mat).reshape(b2,r2,b1,r1)):
        rrbb_list =[]
        for rr, bb in zip(r, b):
            rrrbbb_list = []
            for rrr, bbb in zip(rr, bb):
                rrrbbb_list.append(all(rrr==bbb))
            rrbb_list.append(any(rrrbbb_list))
        rb_list.append(any(rrbb_list))        
    return all(rb_list)
    
similarity_candidates_rdd_60_3 = signature_matrix_rdd_60.map(lambda x: (x[0], And_Or_Or_And(x[1]))) \
    .filter(lambda x: x[1])
similarity_candidates_60_3 = similarity_candidates_rdd_60_3.collect()
print(len(similarity_candidates_60_3))
similarity_candidates_rdd_60_3.take(2)

658018


[((9240801, [205202, 900126, 205201, 100701250, 900241, 169]), True),
 ((9511665, [100700853]), True)]

In [77]:
top_five_similar_pathes = [a[0][1] for a in top_five_similar_pathes_rdd]

[[205202, 900126, 205201, 100701250, 900241, 169],
 [169, 169, 205201, 205202, 900126, 900197, 900241, 103001],
 [100701155, 205201, 900241, 205202, 100701251, 169, 100701250, 141, 169],
 [169, 205202, 205201],
 [900126, 900126, 900126, 169, 900241]]

In [90]:
def check_validity(input):
    input_list = [a[0][1] for a in input]
    return [l in input_list for l in top_five_similar_pathes]

print("validity 20 hyperplanes And_Or: ", check_validity(similarity_candidates))
print("validity 20 hyperplanes Or_And: ", check_validity(similarity_candidates_2))
print("validity 40 hyperplanes And_Or: ", check_validity(similarity_candidates_40))
print("validity 40 hyperplanes Or_And: ", check_validity(similarity_candidates_40_2))
print("validity 60 hyperplanes And_Or: ", check_validity(similarity_candidates_60))
print("validity 60 hyperplanes And_Or_Or_And: ", check_validity(similarity_candidates_60_3))


validity 20 hyperplanes And_Or:  [True, True, True, True, True]
validity 20 hyperplanes Or_And:  [True, True, True, True, True]
validity 40 hyperplanes And_Or:  [True, False, True, False, False]
validity 40 hyperplanes Or_And:  [True, True, True, True, True]
validity 60 hyperplanes And_Or:  [True, False, False, False, False]
validity 60 hyperplanes And_Or_Or_And:  [True, True, True, True, False]
